# Building a Chatbot

In this notebook, I followed along with Cohere's tutorial to build a chatbot using Cohere's Chat endpoint. In this notebook, I built a simple chatbot that can respond to user messages and maintain the context of the conversation.

## Overview

We'll do the following steps:
- **Step 1: Quickstart** - Learn the quickest and easiest way to call the Chat endpoint.
- **Step 2: Adding a preamble** - Steer a chatbot's response toward certain styles, personas, or other characteristics.
- **Step 3: Streaming the response** - Display a chatbot's response incrementally as it is generated, as opposed to waiting for the entire response to be completed.
- **Step 4: Building the Chat History** - Explore two different options for getting the chatbot to maintain the context of the conversation.

## Setup

We'll start by installing the tools we'll need and then importing them.

In [1]:
! pip install cohere -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00


In [2]:
#@title Enable text wrapping in Google Colab

from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [3]:
import uuid
import cohere
import json
from cohere import ChatMessage

Fill in your Cohere API key in the next cell. To do this, begin by [signing up to Cohere](https://os.cohere.ai/) (for free!) if you haven't yet. Then get your API key [here](https://dashboard.cohere.com/api-keys).

In [4]:
# Paste your API key here. Remember to not share publicly
co = cohere.Client("9CxqpeaqJSV7nDGiJKCn5A5F8R9Of66492qh5QXL")

## Step 1: Quickstart

The `message` parameter is the only required parameter for the Chat endpoint.

In our case, we also use the `model` parameter to specify the underlying chat model we want to use. For this, we pick Command R+, Cohere's newest large language model.

We call the endpoint with `co.chat()`, and get the main content of the response from the `text` value.

In [5]:
response = co.chat(message="Hello", model="command-r-plus")
print(response.text)

Hello! How can I help you today?


## Step 2: Adding a preamble

A conversation starts with a system message, or a preamble, to help steer a chatbot’s response toward certain characteristics. In the quickstart example, we didn’t have to define a preamble because a default one was used.

We can define our own preamble with the `preamble` parameter.

In [6]:
response = co.chat(message="Hello",
                   model="command-r-plus",
                   preamble="You are an expert medical doctor. Don't use any greetings.")
print(response.text)

I am an AI medical assistant. How can I help you today?


## Step 3: Streaming the response

Our examples so far generate responses in a non-streamed manner. This means that the endpoint would return a response object only after the model has generated the text in full.

The Chat endpoint also supports streamed responses, where you can display the text incrementally without having to wait for the full completion. To activate it, use `co.chat_stream()` instead of `co.chat()`.

In streaming mode, the endpoint will generate a series of objects. To get the text content, we take objects whose `event_type` is `text-generation`.

In [7]:
stream = co.chat_stream(message="Hello. I'd like to learn about how to cure my sore throat",
                        model="command-r-plus",
                        preamble="You are an expert medical doctor.")

for event in stream:
    if event.event_type == "text-generation":
        print(event.text, end='')

Sure! Here is some information about how to cure a sore throat:

- Rest your voice: Speaking or singing can irritate an already sore throat, so it’s important to rest your voice as much as possible until your throat feels better.

- Drink plenty of fluids: Staying hydrated is important when you’re sick, especially if you have a fever. Drink water, juice, or clear broth to help thin out mucus and keep your throat moist. Avoid caffeine and alcohol, as these can dehydrate you further.

- Gargle with warm salt water: Dissolve 1/2 teaspoon of table salt in a glass of warm water and gargle with it several times a day. This can help reduce swelling and soothe irritation in the throat.

- Use a humidifier: Dry air can irritate a sore throat, so using a humidifier can help add moisture to the air and provide some relief.

- Consider over-the-counter medications: There are a number of medications that can help soothe a sore throat, including:

    - Pain relievers: Nonsteroidal anti-inflammatory

## Step 4: Building the Chat history

At the core of a conversation is a multi-turn dialog between the user and the chatbot. This requires the chatbot to have a “memory” of all the previous turns to maintain the state of the conversation.

### Option 1: Using the conversation history persistence feature

The Chat endpoint supports state management by persisting the conversation history. As a conversation progresses, the endpoint continuously updates the conversation history. This means developers don’t have to deal with the complexity and inconvenience of managing conversation history in their application.

To use this feature, use the `conversation_id` parameter, which is a unique string you assign to a conversation. We'll use the `uuid` library to do this.

Putting everything together, let’s now build a simple chat interface that takes in a user message, generates the chatbot response, automatically updates the conversation history, and repeats these steps until the user quits the conversation.

As described before, in streaming mode, the Chat endpoint generates a series of objects. To get the conversation history, we take the object with `event_type` of `"stream-end"` and save it as a new variable `chat_history`.

In [9]:
# Create a conversation ID
conversation_id = str(uuid.uuid4())

# Define the preamble
preamble = "You are an expert medical doctor"

print('Starting the chat. Type "quit" to end.\n')

while True:

    # User message
    message = input("User: ")

    # Typing "quit" ends the conversation
    if message.lower() == 'quit':
        print("Ending chat.")
        break

    # Chatbot response
    stream = co.chat_stream(message=message,
                            model="command-r-plus",
                            preamble=preamble,
                            conversation_id=conversation_id)

    print("Chatbot: ", end='')

    for event in stream:
        if event.event_type == "text-generation":
            print(event.text, end='')
        if event.event_type == "stream-end":
            chat_history = event.response.chat_history

    print(f"\n{'-'*100}\n")

Starting the chat. Type "quit" to end.

User: I sprained my ankle. What should I do?
Chatbot: If you have sprained your ankle, there are several things you can do to facilitate recovery and prevent further injury:

1. Rest: Take weight off the injured ankle and rest as much as possible. Avoid activities that cause pain or discomfort. Use crutches if necessary to keep weight off the ankle.

2. Ice: Apply ice or a cold pack to the ankle as soon as possible after the injury. This will help reduce swelling and pain. Apply ice for 20 minutes at a time, with at least 20 minutes in between icing sessions. Never apply ice directly to the skin, always wrap the ice pack in a thin towel.

3. Compression: Use a compression bandage to reduce swelling and provide support. Start at the farthest point on the foot and wrap the bandage snugly around the ankle and foot, moving upwards. Be careful not to wrap too tightly, as this can restrict blood flow.

4. Elevation: Keep your ankle elevated above the l

Next, we print the full conversation history.

In [10]:
for chat in chat_history:
    print(chat)

message='I sprained my ankle. What should I do?' tool_calls=None role='USER' response_id='266f1988-e250-46fc-83e9-dcf5cd86ac23' generation_id='ce41c0df-603d-4bb0-a303-01ff31c541f0'
message='If you have sprained your ankle, there are several things you can do to facilitate recovery and prevent further injury:\n\n1. Rest: Take weight off the injured ankle and rest as much as possible. Avoid activities that cause pain or discomfort. Use crutches if necessary to keep weight off the ankle.\n\n2. Ice: Apply ice or a cold pack to the ankle as soon as possible after the injury. This will help reduce swelling and pain. Apply ice for 20 minutes at a time, with at least 20 minutes in between icing sessions. Never apply ice directly to the skin, always wrap the ice pack in a thin towel.\n\n3. Compression: Use a compression bandage to reduce swelling and provide support. Start at the farthest point on the foot and wrap the bandage snugly around the ankle and foot, moving upwards. Be careful not to 

### Option 2: Managing the conversation history yourself

If you opt not to use the endpoint’s conversation history persistence feature, you can use the `chat_history` parameter to manage the conversation history yourself.

In [11]:
# Initialize the chat history
chat_history = []

# Define the preamble
preamble = "You are an expert medical doctor"

print('Starting the chat. Type "quit" to end.\n')

while True:

    # User message
    message = input("User: ")

    # Typing "quit" ends the conversation
    if message.lower() == 'quit':
        print("Ending chat.")
        break

    # Chatbot response
    stream = co.chat_stream(message=message,
                            model="command-r-plus",
                            preamble=preamble,
                            chat_history=chat_history)

    chatbot_response = ""
    print("Chatbot: ", end='')

    for event in stream:
        if event.event_type == "text-generation":
            print(event.text, end='')
            chatbot_response += event.text
    print("\n")

    # Add to chat history
    chat_history.extend(
        [ChatMessage(role="USER", message=message),
         ChatMessage(role="CHATBOT", message=chatbot_response)]
    )

Starting the chat. Type "quit" to end.

User: I have a really bad sore throat. I can barely talk. What should I do? 
Chatbot: I'm sorry to hear that you're dealing with a sore throat. Here are some recommendations to help you feel better:

1. Rest your voice: Try to rest your voice as much as possible. Avoid talking or whispering, as this can strain your vocal cords further. Use non-verbal communication methods when necessary.

2. Stay hydrated: Drink plenty of fluids, particularly warm liquids such as tea with honey, lemon water, or warm broth. This will help soothe your throat and keep your mucous membranes moist.

3. Gargle with salt water: Mix 1/2 teaspoon of salt in a glass of warm water and gargle with it several times a day. This can help reduce swelling and discomfort in your throat.

4. Use throat lozenges: Suck on throat lozenges or hard candies. Look for lozenges that contain menthol, which can provide a cooling sensation and help numb the throat.

5. Humidify the air: Use a

We confirm that `chat_history` is consistent with the conversation above.

In [12]:
for chat in chat_history:
    print(chat)

message='I have a really bad sore throat. I can barely talk. What should I do? ' tool_calls=None role='USER'
message="I'm sorry to hear that you're dealing with a sore throat. Here are some recommendations to help you feel better:\n\n1. Rest your voice: Try to rest your voice as much as possible. Avoid talking or whispering, as this can strain your vocal cords further. Use non-verbal communication methods when necessary.\n\n2. Stay hydrated: Drink plenty of fluids, particularly warm liquids such as tea with honey, lemon water, or warm broth. This will help soothe your throat and keep your mucous membranes moist.\n\n3. Gargle with salt water: Mix 1/2 teaspoon of salt in a glass of warm water and gargle with it several times a day. This can help reduce swelling and discomfort in your throat.\n\n4. Use throat lozenges: Suck on throat lozenges or hard candies. Look for lozenges that contain menthol, which can provide a cooling sensation and help numb the throat.\n\n5. Humidify the air: Use

And with that, we have built a simple chatbot that can respond to user messages and maintain the context of the conversation.